In [1]:
import tweepy
import sys
import json
import urllib
import urllib.request
import twitter
import os

In [2]:
### In order to access to the Twitter API, you need to have a developer account.
t = twitter.Twitter(auth=twitter.OAuth("your_consumer_key", "your_consumer_secret", "your_access_token", "your_access_token_secret"))

In [6]:
### This code simply downloads the profile images of the users
classifiedUsersPath="~/manuallyClassifiedUsers_2001_3000.json"
usersPath="~/userss.json"

classifiedUsers=open(classifiedUsersPath)
users=open(usersPath)

notFounds = []
for classifiedUser in classifiedUsers:
    classifiedUser_object = json.loads(classifiedUser) #gets the first line from manuallyClassifiedUsers..
    userName = classifiedUser_object["field1"]
    userType = classifiedUser_object["field0"]
    imgName = ''
    profile_img_url = ''
    profile_img_url, imgName = getProfileImgs(userName, userType, profile_img_url, imgName)            
    try:
        urllib.request.urlretrieve(profile_img_url, "~/marujana_imgs/%s" % imgName)    
    except:
        notFounds.append(userName)
        print(profile_img_url)
        print('No {} username or id found for {}'.format(userType, userName))        

In [5]:
### getProfileImgs method looks up the userName passed in the function
### if it is a valid username it finds the associated profile img url for the user.
### If it is not a valid username (user might have changed their screenName),
### the method then checks the id of that user in userss.json file that contains all info about all users.
### If it finds the user on Twitter based on user's id, then again it gets the profile img url of the user to be downloaded
### If neither is the case, it simply returns a fake profile img url.
### getProfileImgs method returns profile_img_url and imgName info at the end

usersPath="~/userss.json"
users=open(usersPath)
def getProfileImgs(userName, userType, profile_img_url, imgName):
    try:        
        info = t.users.lookup(screen_name=userName)
        result = json.dumps(info)
        lobj = json.loads(result) #type is list
        profile_img_url = lobj[0]["profile_image_url_https"]
        ext = 'jpg'#profile_img_url.split('.')[-1]
        imgName = (userName + '.' + userType + '.' + ext)
        #return profile_img_url
    except:
        for user in users:
            user_object = json.loads(user)
            if user_object['tweet']['user']['screenName'] == userName:
                userId = user_object['tweet']['user']['id']
                try:                    
                    info = t.users.lookup(user_id=userId)
                    result = json.dumps(info)
                    lobj = json.loads(result) #type is list
                    profile_img_url = lobj[0]["profile_image_url_https"]
                    ext = 'jpg'#profile_img_url.split('.')[-1]
                    imgName = (userName + '.' + userType + '.' + ext)
                    #print('here in try')                 
                except:                    
                    profile_img_url = 'https://pbs.twimg.com/invalid_anormalUrl.jpg'
                    imgName = 'error.jpg'
                    #print(profile_img_url)
                break            
    return profile_img_url, imgName